# Clustering Crypto

In [410]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [411]:
file_path="./crypto_data.csv"
df=pd.read_csv(file_path, index_col=False)


In [412]:
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [413]:

df=df.set_index("Unnamed: 0")
df.index.name = None

In [414]:
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [415]:
# file_path="./crypto_data.csv"
# df1=pd.read_csv(file_path)
# df1.set_index("Unnamed: 0")

In [416]:
# df1.set_index(["Unnamed: 0","CoinName","Algorithm","IsTrading","ProofType","TotalCoinsMined","TotalCoinSupply"])
# df1.head()

In [417]:
df["IsTrading"].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [418]:
# Keep all the cryptocurrencies that are being traded.
df=df[df["IsTrading"]==True]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [419]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [420]:
# Keep all the cryptocurrencies that have a working algorithm.

#--There is no null Algorithem in ccolumn Algorithem 



In [421]:
# Remove the "IsTrading" column. 
df_cleaned=df.drop(columns="IsTrading")
df_cleaned.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [422]:
# Remove rows that have at least 1 null value.
df_cleaned=df_cleaned.dropna()

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 685 entries, 42 to XBC
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         685 non-null    object 
 1   Algorithm        685 non-null    object 
 2   ProofType        685 non-null    object 
 3   TotalCoinsMined  685 non-null    float64
 4   TotalCoinSupply  685 non-null    object 
dtypes: float64(1), object(4)
memory usage: 32.1+ KB


In [423]:
# Keep the rows where coins are mined.
coin_df=df_cleaned[df_cleaned["TotalCoinsMined"]>0]
coin_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [424]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df=pd.DataFrame(coin_df.CoinName)
coin_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [425]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
coin_df=coin_df.drop(columns="CoinName")
coin_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [426]:
# Use get_dummies() to create variables for text features.
coin_df_ecoded=pd.get_dummies(coin_df, columns=["ProofType","Algorithm"])
coin_df_ecoded.head()

,TotalCoinsMined,TotalCoinSupply,ProofType_DPOS,ProofType_DPoS,ProofType_HPoW,ProofType_LPoS,ProofType_POBh,ProofType_PoA,ProofType_PoC,ProofType_PoS,...,Algorithm_Tribus,Algorithm_VBFT,Algorithm_VeChainThor Authority,Algorithm_X11,Algorithm_X11GOST,Algorithm_X13,Algorithm_X14,Algorithm_X15,Algorithm_X16R,Algorithm_XEVAN
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [427]:
# Standardize the data with StandardScaler().
coin_df_scaled=StandardScaler().fit_transform(coin_df_ecoded)
coin_df_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.08703883,
        -0.08703883, -0.10680283],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.08703883,
        -0.08703883, -0.10680283],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.08703883,
        -0.08703883, -0.10680283],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.08703883,
        -0.08703883, -0.10680283],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.08703883,
        -0.08703883, -0.10680283],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.08703883,
        -0.08703883, -0.10680283]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [428]:
# Using PCA to reduce dimension to three principal components.
pca=PCA(n_components=3)
coin_pca = pca.fit_transform(coin_df_scaled)
coin_pca

array([[-0.34491617,  0.98395874, -0.5585212 ],
       [-0.32822089,  0.9842504 , -0.5589633 ],
       [ 2.32063274,  1.6678586 , -0.69582363],
       ...,
       [ 0.33385557, -2.27136169,  0.42183362],
       [-0.1463454 , -2.01960051,  0.3242549 ],
       [-0.29449806,  0.82726377, -0.24949619]])

In [429]:
pca.explained_variance_ratio_

array([0.02793093, 0.02140634, 0.02050373])

In [430]:
# Create a DataFrame with the three principal components.
coin_pca_df= pd.DataFrame(coin_pca, columns=["pca1","pca2","pc3"], index=coin_df.index)
coin_pca_df.head()

,pca1,pca2,pc3
42,-0.344916,0.983959,-0.558521
404,-0.328221,0.984250,-0.558963
1337,2.320633,1.667859,-0.695824
BTC,-0.138978,-1.317589,0.204511
ETH,-0.140767,-1.989244,0.435046


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [431]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1, 11))
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(coin_pca_df)
    inertia.append(km.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [432]:
# Initialize the K-Means model.
kmodel=KMeans(n_clusters=4, random_state=42)

# Fit the model
kmodel.fit(coin_pca)

# Predict clusters
prediction=kmodel.predict(coin_pca)

print(prediction)

[1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1
 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0
 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0
 1 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1
 1 0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1
 0 0 1 0 0 0 0 1 3 3 0 0 0 1 3 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0
 1 1 1 1 0 1 1 0 1 1 0 3 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1
 0 0 0 1 1 1 0 1 0 1 0 1 

In [433]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

combined = coin_df.join(coin_pca_df, how='inner')
# df_y = pd.DataFrame(prediction, columns=['Cluster'])
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
combined["CoinName"]=coin_name_df["CoinName"]
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustred_df=combined.copy()
clustred_df["Class"]=prediction
clustred_df

# Print the shape of the clustered_df
# print(clustered_df.shape)
# clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pca1,pca2,pc3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.344916,0.983959,-0.558521,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.328221,0.984250,-0.558963,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.320633,1.667859,-0.695824,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.138978,-1.317589,0.204511,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.140767,-1.989244,0.435046,Ethereum,0
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.451385,0.696496,-0.008091,ZEPHYR,1
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.342956,0.983865,-0.558552,Gapcoin,1
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.333856,-2.271362,0.421834,Beldex,0
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.146345,-2.019601,0.324255,Horizen,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [434]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustred_df,
    x="pca1",
    y="pca2",
    z="pc3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [440]:
# Create a table with tradable cryptocurrencies.
tradable_tb=clustred_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinsMined","TotalCoinSupply","Class"], sortable=True, selectable=True)
tradable_tb

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class]

In [436]:
# Print the total number of tradable cryptocurrencies.
tradable_coin=tradable_tb.rows-1

print(f"There are {tradable_coin} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [437]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X=clustred_df[["TotalCoinSupply","TotalCoinsMined"]]
scaler=MinMaxScaler()
scaled_x=scaler.fit_transform(X)
scaled_x

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [438]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_x_df=pd.DataFrame(scaled_x, columns=["TotalCoinSupply","TotalCoinsMined"],index=clustred_df.index)
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_x_df["CoinName"]=clustred_df["CoinName"]
scaled_x_df["Class"]=clustred_df["Class"]

scaled_x_df

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000e+00,42 Coin,1
404,5.320000e-04,1.065855e-03,404Coin,1
1337,3.141593e-01,2.957551e-02,EliteCoin,1
BTC,2.100000e-05,1.810842e-05,Bitcoin,0
ETH,0.000000e+00,1.087731e-04,Ethereum,0
...,...,...,...,...
ZEPH,2.000000e-03,2.020225e-03,ZEPHYR,1
GAP,2.500000e-04,1.508199e-05,Gapcoin,1
BDX,1.400223e-03,9.901351e-04,Beldex,0
ZEN,2.100000e-05,7.370282e-06,Horizen,0


In [439]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_x_df.hvplot.scatter(
    x="TotalCoinSupply",
    y="TotalCoinsMined",
    hover_cols=["Class"],
    by="Class"
)



:NdOverlay   [Class]
   :Scatter   [TotalCoinSupply]   (TotalCoinsMined,Class)